In [6]:
import mysql.connector
from mysql.connector import Error
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests

# Hàm kết nối cơ sở dữ liệu
def create_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",  # Để trống nếu không có mật khẩu
            database="thongbao"
        )
        if connection.is_connected():
            print("Kết nối MySQL thành công")
        return connection
    except Error as e:
        print(f"Lỗi khi kết nối MySQL: {e}")
        return None

# Hàm lưu thông báo vào MySQL
def save_announcement_to_db(connection, table, date, title, content):
    try:
        cursor = connection.cursor()
        # Sử dụng tên bảng động
        insert_query = f"INSERT INTO {table} (ngay, tieude, noidung) VALUES (%s, %s, %s)"
        data_tuple = (date, title, content)
        cursor.execute(insert_query, data_tuple)
        connection.commit()
        print(f"Thông báo '{title}' đã được lưu vào bảng {table}.")
    except Error as e:
        print(f"Lỗi khi lưu thông báo: {e}")

# Hàm gửi dữ liệu thông báo lên Web API
def send_announcement_to_api(api_url, date, title, content):
    try:
        payload = {
            'date': date,
            'title': title,
            'content': content
        }
        response = requests.post(api_url, json=payload)
        print(f"Payload sent: {payload}")  # Debug thông tin payload
        print(f"Response Status Code: {response.status_code}")  # Debug mã trạng thái phản hồi
        print(f"Response Text: {response.text}")  # Debug văn bản phản hồi

        if response.status_code == 200:
            print(f"Thông báo '{title}' đã được gửi lên Web API thành công.")
        else:
            print(f"Lỗi khi gửi thông báo lên Web API: {response.status_code} - {response.text}")
    except requests.RequestException as e:
        print(f"Lỗi khi gửi yêu cầu đến Web API: {e}")


# Hàm phân tích và lưu thông báo
def extract_announcements(soup, tab_id, connection, table, api_url):
    tab_content = soup.find(id=tab_id)
    if tab_content:
        announcements = tab_content.select("div.tbBox")
        if not announcements:
            print(f"Không tìm thấy thông báo nào trong tab với id: {tab_id}")
        for announcement in announcements:
            caption = announcement.select_one("div.tbBoxCaption")
            content = announcement.select_one("div.tbBoxContent")

            if caption and content:
                spans = caption.find_all("span")
                if len(spans) >= 2:
                    date = spans[0].text.strip()
                    title = spans[1].text.strip()
                else:
                    date = "No date"
                    title = "No title"

                # Xử lý các thẻ <a> trong nội dung
                for a_tag in content.find_all("a"):
                    href = a_tag.get("href", "")
                    a_tag.string = f"{a_tag.text.strip()} ({href})"

                announcement_content = content.text.strip()

                # Kiểm tra kết nối trước khi lưu
                if connection and connection.is_connected():
                    save_announcement_to_db(connection, table, date, title, announcement_content)
                else:
                    print("Mất kết nối đến MySQL!")

                # Gửi thông báo lên Web API
                send_announcement_to_api(api_url, date, title, announcement_content)

                print(f"Ngày: {date}")
                print(f"Tiêu đề: {title}")
                print(f"Nội dung: {announcement_content}")
                print("-" * 40)
    else:
        print(f"Không tìm thấy tab với id: {tab_id}")

# Cấu hình trình duyệt Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Chạy Chrome ở chế độ headless
service = Service(ChromeDriverManager().install())

# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # Tạo kết nối cơ sở dữ liệu
    connection = create_connection()

    # Tải trang web
    url = 'http://sv.dut.udn.vn/'
    driver.get(url)

    # Tạo WebDriverWait
    wait = WebDriverWait(driver, 10)  # Chờ tối đa 10 giây

    # URL của Web API
    api_url = 'http://127.0.0.1:5000/announcements'  # Thay đổi URL API theo yêu cầu

    # Duyệt qua các tab
    tab_info = {
        "tabs_PubTB-divT0": "thongbaochung1",  # Tab thông báo chung
        "tabs_PubTB-divT1": "thongbaoLHP"     # Tab lớp học phần
    }

    for tab_id, table in tab_info.items():
        try:
            # Nhấp vào tab
            tab_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'li[aria-controls="{tab_id}"] a')))
            tab_link.click()
            print("-" * 50)
            print(f"Nhấp vào tab với id: {tab_id}")
            print("-" * 50)

            # Chờ nội dung tab hiển thị
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f'#{tab_id} div.tbBox')))

            # Phân tích HTML bằng BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Xử lý và lưu thông báo vào bảng thích hợp
            extract_announcements(soup, tab_id, connection, table, api_url)

        except Exception as e:
            print(f"Lỗi khi xử lý tab với id: {tab_id}: {e}")

finally:
    # Đóng trình duyệt và kết nối
    driver.quit()
    if connection and connection.is_connected():
        connection.close()
        print("Đã đóng kết nối MySQL.")


Kết nối MySQL thành công
--------------------------------------------------
Nhấp vào tab với id: tabs_PubTB-divT0
--------------------------------------------------
Thông báo 'Lịch thi đợt thi tiếng Anh chuẩn đầu ra (VSTEP) ngày 15/9/2024 do ĐH Ngoại ngữ tổ chức' đã được lưu vào bảng thongbaochung1.
Payload sent: {'date': '12/09/2024:', 'title': 'Lịch thi đợt thi tiếng Anh chuẩn đầu ra (VSTEP) ngày 15/9/2024 do ĐH Ngoại ngữ tổ chức', 'content': '- Sinh viên xem thông báo tại đây (https://dut.udn.vn/Phong/QualityAssurance/Thongbao/id/9661)'}
Response Status Code: 404
Response Text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

Lỗi khi gửi thông báo lên Web API: 404 - <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually pl